In [1]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [2]:
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

# Load pre-trained InceptionResNetV2 model
facenet_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

# Function to get face embeddings from an image
def get_face_embedding(image):
    image = cv2.resize(image, (299, 299))  # Updated size
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    embedding = facenet_model.predict(image)
    return embedding.flatten()

# Load dataset
data = []
labels = []
dataset_path = 'E:/GitHub-rzn/Poopy-or-Not-Poopy/Dataset'  # Path to your dataset (folders per person)

for person_folder in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_folder)
    for image_file in os.listdir(person_path):
        image_path = os.path.join(person_path, image_file)
        image = cv2.imread(image_path)
        if image is None:
            continue
        face_embedding = get_face_embedding(image)
        data.append(face_embedding)
        labels.append(person_folder)

# Convert data and labels to arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.3, random_state=42)

# Experiment with different values of k
best_k = 3
best_score = 0

for k in range(1, 11):  # Try k values from 1 to 10
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(f"Accuracy with k={k}: {score * 100:.2f}%")
    
    # If this k gives a better score, save it
    if score > best_score:
        best_k = k
        best_score = score
        best_knn = knn

print(f"Best k: {best_k} with accuracy: {best_score * 100:.2f}%")

# Save the best model and label encoder
import pickle
os.makedirs('model', exist_ok=True)
with open('model/knn_classifier.pkl', 'wb') as f:
    pickle.dump(best_knn, f)
with open('model/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# Evaluate the model
y_pred = best_knn.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s